In [1]:
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
import nltk
nltk.download('stopwords')
#nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rsantizo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords


In [7]:
import spacy

In [8]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [11]:
stopwords.words('spanish')

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [12]:
df = pd.read_json('../data/raw/newsgroups.json')
print(df.target_names.unique())
df

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [13]:
df.loc[0,'content']

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [14]:
# Convertir a una lista
data = df.content.values.tolist()
pprint(data[:1])

["From: lerxst@wam.umd.edu (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [15]:
# Eliminar emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

["From: (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [16]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

["From: (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: "
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [17]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [18]:
def sent_to_words(sentences):
    for sentence in sentences:
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [19]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10)
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [20]:
print(bigram[data_words[0]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting', 'host_rac', 'wam_umd', 'edu_organization', 'university', 'of', 'maryland_college', 'park', 'lines', 'was_wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten_me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports_car', 'looked', 'to', 'be', 'from', 'the', 'late_early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in_addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or_whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please_mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [21]:
print(trigram[bigram[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host_rac', 'wam_umd_edu_organization', 'university', 'of_maryland_college', 'park_lines', 'was_wondering_if', 'anyone_out', 'there', 'could', 'enlighten_me', 'on', 'this', 'car', 'saw', 'the', 'other_day', 'it', 'was', 'door', 'sports_car', 'looked', 'to', 'be', 'from', 'the', 'late_early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in_addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest_of', 'the', 'body', 'this', 'is', 'all', 'know', 'if_anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or_whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please_mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [22]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting', 'host_rac', 'wam_umd', 'edu_organization', 'university', 'of', 'maryland_college', 'park', 'lines', 'was_wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten_me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports_car', 'looked', 'to', 'be', 'from', 'the', 'late_early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in_addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or_whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please_mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']
['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'thi

In [23]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [25]:
data_words_nostops[0]

['wheres',
 'thing',
 'car',
 'nntp',
 'posting',
 'host',
 'rac',
 'wam',
 'umd',
 'organization',
 'university',
 'maryland',
 'college',
 'park',
 'lines',
 'wondering',
 'anyone',
 'could',
 'enlighten',
 'car',
 'saw',
 'day',
 'door',
 'sports',
 'car',
 'looked',
 'late',
 'early',
 'called',
 'bricklin',
 'doors',
 'really',
 'small',
 'addition',
 'front',
 'bumper',
 'separate',
 'rest',
 'body',
 'know',
 'anyone',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'production',
 'car',
 'made',
 'history',
 'whatever',
 'info',
 'funky',
 'looking',
 'car',
 'please',
 'mail',
 'thanks',
 'il',
 'brought',
 'neighborhood',
 'lerxst']

In [26]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [27]:
data_words_bigrams[0]

['wheres',
 'thing',
 'car',
 'nntp_posting',
 'host_rac',
 'wam_umd',
 'organization_university',
 'maryland_college',
 'park',
 'lines',
 'wondering',
 'anyone',
 'could',
 'enlighten',
 'car',
 'saw',
 'day',
 'door',
 'sports_car',
 'looked',
 'late_early',
 'called',
 'bricklin',
 'doors',
 'really',
 'small',
 'addition',
 'front_bumper',
 'separate',
 'rest',
 'body',
 'know',
 'anyone',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'production',
 'car',
 'made',
 'history',
 'whatever',
 'info',
 'funky',
 'looking',
 'car',
 'please_mail',
 'thanks',
 'il',
 'brought',
 'neighborhood',
 'lerxst']

In [28]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host_rac', 'wam_umd', 'maryland_college', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'door', 'look', 'late_early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'thank', 'bring', 'neighborhood']]


### <span style="color:blue">HASTA AQUI PREPROCESAMIENTO</span> 


In [29]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [30]:
for key, value in id2word.items():
    print(key, value)

0 addition
1 body
2 bring
3 call
4 car
5 could
6 door
7 engine
8 enlighten
9 funky
10 history
11 host_rac
12 info
13 know
14 late_early
15 line
16 look
17 make
18 maryland_college
19 model
20 name
21 neighborhood
22 nntp_poste
23 park
24 production
25 really
26 rest
27 see
28 separate
29 small
30 spec
31 thank
32 thing
33 wam_umd
34 where
35 wonder
36 year
37 acceleration
38 adapter
39 add
40 answer
41 attain
42 base
43 brave
44 brief
45 card
46 clock
47 detail
48 do
49 especially
50 experience
51 fair
52 final
53 floppy
54 floppy_disk
55 functionality
56 guy_kuo
57 heat_sink
58 hour
59 knowledge
60 message
61 network
62 next
63 number
64 oscillator
65 per_day
66 please_send
67 poll
68 procedure
69 rate
70 report
71 request
72 share
73 si_clock
74 soul
75 speed
76 summarize
77 summary
78 top
79 two_day
80 upgrade
81 usage
82 access
83 active
84 actually
85 advance
86 anymore
87 appearence
88 back
89 be
90 bite
91 breifly
92 bunch
93 computer
94 computer_network
95 conviction
96 daily
9

1327 software_inc
1328 unsafe
1329 wall
1330 worry
1331 acquire
1332 catch
1333 cheaply
1334 customer
1335 damage
1336 deck
1337 depreciate
1338 deter
1339 discourage
1340 discouragement
1341 disheartened
1342 face
1343 greatly
1344 legitimate
1345 local
1346 nee
1347 notify
1348 originally
1349 pullout
1350 purpose
1351 refer
1352 register
1353 remarkably
1354 removeable
1355 replace
1356 replacement
1357 shop
1358 sleave
1359 source
1360 stand
1361 steal
1362 theive
1363 thus
1364 unhappy
1365 unit
1366 useless
1367 warranty
1368 acne
1369 advice
1370 anti
1371 appearance
1372 bury
1373 cap
1374 chemist
1375 chin
1376 counter
1377 cradle
1378 dandruff
1379 diet
1380 disfigure
1381 doctor
1382 escalate
1383 father
1384 flogging
1385 greasy
1386 hairline
1387 improve
1388 incline
1389 kid
1390 malady
1391 morale
1392 neighbour
1393 odd
1394 ointment
1395 overdose
1396 pharmacist
1397 prescription
1398 product_calle
1399 scaliness
1400 scalp
1401 senstitive
1402 serious
1403 severe
1404

2342 synchronous
2343 telescope
2344 third
2345 threefold
2346 tilt
2347 tiny
2348 titan
2349 tour
2350 track
2351 trajectory
2352 transit
2353 transport
2354 traverse
2355 unfurl
2356 unique
2357 universe
2358 upcome
2359 upcoming
2360 upper_atmosphere
2361 vaporize
2362 variable
2363 variation
2364 various
2365 wave
2366 wavelength
2367 welcome
2368 wind
2369 world_expire
2370 anyone_know
2371 cs_dalhousie
2372 electrical
2373 math_stat
2374 netter
2375 ns_canada
2376 automate
2377 bill
2378 conference
2379 cr
2380 event
2381 extenstion
2382 modify
2383 molecular_simulation
2384 motif
2385 pl_line
2386 playback
2387 quit
2388 regression_teste
2389 rerun
2390 server
2391 sharing
2392 tar
2393 test
2394 testing
2395 verification
2396 xtrap
2397 xtrapv
2398 notice
2399 break
2400 cd
2401 diversity
2402 following_cds
2403 heart
2404 sale
2405 additional_information
2406 automobile
2407 depletion
2408 greatly_appreciate
2409 guilty
2410 partially
2411 ancient
2412 apply
2413 atheism
2414 

3073 portal_communication
3074 ville_walveranta
3075 brag
3076 consideration
3077 cross
3078 decent
3079 flashy
3080 gruffness
3081 guru
3082 halfway
3083 inline
3084 liter
3085 mid
3086 noise
3087 noisy
3088 organization_clearer
3089 recommendation
3090 styling
3091 thumb
3092 aware
3093 basicly
3094 clustering
3095 connector
3096 consent
3097 dissent
3098 draft
3099 impedance
3100 implementor
3101 internal
3102 meter
3103 reflection
3104 scsi_cable
3105 scsi_device
3106 silence
3107 silent
3108 stub
3109 violate
3110 chase
3111 compile
3112 compiler
3113 ctrltest
3114 differ
3115 directory
3116 dont_think
3117 employer
3118 gotcha
3119 hard_code
3120 hitch
3121 impossible
3122 initialize
3123 int
3124 menu
3125 overload
3126 overloaded
3127 piece
3128 portability
3129 properly
3130 void
3131 whole
3132 arbitrary
3133 char
3134 constrain
3135 convex
3136 dresden_de
3137 extrude
3138 generic
3139 hull
3140 mirror
3141 picture
3142 placement
3143 polygonal
3144 postscript
3145 pov
3146 

3824 slice
3825 solidarity
3826 spiral
3827 succumb
3828 terrified
3829 three_soldier
3830 tonight
3831 unspeakable
3832 utopic
3833 victim
3834 violence
3835 whatsoever
3836 effectiveness
3837 hlv
3838 large_amount
3839 laser
3840 launch_hlv
3841 launcher
3842 pad
3843 pricetag
3844 sized
3845 toronto_zoology
3846 arrangement
3847 billion
3848 bud
3849 cooperative
3850 supercede
3851 ability
3852 athletic
3853 barry_bond
3854 career
3855 complain
3856 computer_inc
3857 decline
3858 entertainer
3859 exception
3860 gondolin
3861 million_dollar
3862 near
3863 pirate
3864 psbdn
3865 remeber
3866 take_away
3867 hi_im
3868 studio
3869 alias
3870 change_license
3871 computer_service
3872 couldnt_find
3873 wanted
3874 behaviour
3875 condem
3876 disproof
3877 familiar
3878 long_time
3879 pertain
3880 rephrase
3881 beer
3882 bread
3883 breakfast
3884 calcium
3885 cereal
3886 chelate
3887 chelated
3888 chemistry
3889 chromium
3890 compatibility
3891 diagnose
3892 discuss
3893 excess
3894 food
38

...

32724 dissonant
32725 macadamia
32726 prestigious
32727 manhood
32728 misc_comp
32729 broacast
32730 gf
32731 colligation
32732 cryptowrangler
32733 valvoline
32734 perfidious
32735 tonne
32736 halogen
32737 reinsert
32738 attatche
32739 tablehockey
32740 akgun
32741 bloodless
32742 clandestine
32743 doest
32744 durue
32745 duodock
32746 en_sc
32747 powerup
32748 standby
32749 bustingly
32750 mincer
32751 adue
32752 caluse
32753 ceastation
32754 criticeze
32755 curcuit
32756 distinquish
32757 dubiously
32758 exhaustively
32759 freethinker
32760 impossiblity
32761 paradoxical
32762 pinprick
32763 presuppostion
32764 presuppostiton
32765 presuption
32766 unforunate
32767 annette
32768 chapstick
32769 parched
32770 waxy
32771 button_press
32772 ceric
32773 checkoff
32774 clx
32775 cslip
32776 dxwm
32777 examples
32778 exug
32779 fulton
32780 gatekeeper_dec
32781 gatewaye
32782 george_ferguson
32783 identically
32784 imperial_college
32785 interest_groups
32786 latency
32787 lzw
32788 nnn


33476 etiquitte
33477 hauling
33478 keel
33479 cookout
33480 dissappear
33481 libility
33482 manhole
33483 onramp
33484 outhouse
33485 outlandishly
33486 purr
33487 threate
33488 usuall
33489 abvious
33490 autoexec_bat
33491 mscdex
33492 apple_com
33493 earl_wallace
33494 oc
33495 accountant
33496 ayurveda
33497 bloodlette
33498 cupping
33499 gr
33500 nosode
33501 satisfac
33502 unreported
33503 benched
33504 physics_laboratory
33505 berlin_de
33506 mactool
33507 shtick
33508 blinker
33509 k
33510 rtn
33511 categorical
33512 polkadoted
33513 abekas
33514 feather
33515 host_cc_sysk
33516 rol
33517 kitten
33518 swith
33519 clipper_initiative
33520 track_record
33521 disengage
33522 linchpin
33523 dozer
33524 stingray
33525 supercar
33526 qmpro
33527 acm_rpi
33528 nutek
33529 flunitrazepam
33530 librium
33531 sedative
33532 tranquillizer
33533 valium
33534 hangar
33535 rudder
33536 wingtip
33537 chryco
33538 desiscion
33539 plymouth_sundance
33540 dblspaced
33541 eddie_murray
33542 famer


34101 geoff_sanderson
34102 malcontent
34103 subpar
34104 team_biggest
34105 underrate
34106 twg
34107 unstick
34108 muted
34109 proteced
34110 boston_university
34111 cannot_blame
34112 rejoin
34113 spar
34114 fortune_guzzler
34115 escrow_mykotronx
34116 essentialy
34117 hetersexual
34118 misteriously
34119 arogant
34120 inflexible
34121 permision
34122 berryhill
34123 deion
34124 persuaded
34125 schuerholz
34126 tiddleywink
34127 whiffer
34128 whiteydal
34129 procomm_plu
34130 kyanite
34131 perscription
34132 fowl
34133 healthily
34134 landfill
34135 relgious
34136 smokestack
34137 unliveable
34138 inbred
34139 cheers_valentine
34140 dont_blame
34141 rebuttle
34142 spineless
34143 drawnbutton
34144 lymb
34145 ucrengr
34146 cellar
34147 duran_duran
34148 engdahl
34149 congested
34150 beltway
34151 dingbat
34152 impatiently
34153 yahoo
34154 yeild
34155 yoyo
34156 polymath
34157 devaluate
34158 tb
34159 ingress
34160 melencholic
34161 neuroticism
34162 sanguine
34163 amerikan
34164 ent

34675 amusingly
34676 lr
34677 anise
34678 sluggard
34679 toil
34680 rec_audio
34681 crossdominance
34682 brader_write
34683 ottomenu
34684 technische_universitaet
34685 miller_write
34686 supras
34687 irreplaceable
34688 badassbiker
34689 crotch
34690 frontline
34691 reestablishment
34692 saudi_arabia
34693 imagewriter
34694 backwater
34695 domesticated
34696 eugenist
34697 eugenize
34698 gene_pool
34699 microbiological
34700 progeny
34701 transmissible
34702 vulcanism
34703 affirmatively
34704 videomode
34705 respone
34706 gregory_humphreys
34707 proficiently
34708 veridical
34709 inflateble
34710 personnelly
34711 sealing
34712 apathetic
34713 emancipate
34714 emancipation
34715 pasha
34716 talaat
34717 tehlerian
34718 kiloton
34719 garnet
34720 critcle
34721 amphibious
34722 diorama
34723 enacter
34724 ammount
34725 babygirl
34726 botlle
34727 oldd
34728 raajij
34729 rouhy
34730 solider
34731 vehicule
34732 chumbo
34733 firts
34734 hearken
34735 rollout
34736 tirelessly
34737 flamm

35854 hw_and_sw
35855 glassner
35856 reencrypted
35857 karate
35858 mallard
35859 raptor
35860 tailed
35861 cheapish
35862 fiddling
35863 arrese
35864 deference
35865 overide
35866 rjl
35867 irz
35868 shephard
35869 celebate
35870 link_hudson
35871 reprocess
35872 coverare
35873 athenian
35874 campaigner
35875 mediteranean
35876 partione
35877 rift
35878 western_thrace
35879 yugoslav
35880 gorgon
35881 bih_harvard
35882 dilatation
35883 pullover
35884 reupholster
35885 uni
35886 easter_what
35887 crd
35888 echo_set
35889 manipluate
35890 glandular
35891 penicillin
35892 sides
35893 tonsil
35894 tonsilitis
35895 unassigned
35896 doctrinaire
35897 front_panel
35898 four_months
35899 babes
35900 fingers
35901 magnetism
35902 margarita
35903 noser
35904 perpindicular
35905 pruned
35906 spinning
35907 turntable
35908 organization_csiro
35909 anticonvulsant
35910 actuator
35911 thermal_recalibration
35912 comming
35913 extortionist
35914 renaissance
35915 sharpshooter
35916 cleanseth
35917 w

In [31]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 4), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


In [32]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 4),
  ('could', 1),
  ('door', 2),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host_rac', 1),
  ('info', 1),
  ('know', 1),
  ('late_early', 1),
  ('line', 1),
  ('look', 2),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('thank', 1),
  ('thing', 1),
  ('wam_umd', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [34]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.045*"people" + 0.028*"say" + 0.015*"state" + 0.014*"fact" + 0.012*"man" + '
  '0.012*"right" + 0.012*"law" + 0.011*"live" + 0.011*"believe" + '
  '0.010*"person"'),
 (1,
  '0.122*"gun" + 0.044*"weapon" + 0.043*"firearm" + 0.040*"crime" + '
  '0.038*"fire" + 0.038*"police" + 0.034*"trial" + 0.021*"license" + '
  '0.021*"carry" + 0.021*"minority"'),
 (2,
  '0.016*"year" + 0.015*"car" + 0.014*"new" + 0.013*"back" + 0.013*"buy" + '
  '0.012*"first" + 0.011*"old" + 0.011*"power" + 0.010*"high" + 0.010*"price"'),
 (3,
  '0.206*"host" + 0.073*"screen" + 0.070*"image" + 0.063*"nntp_poste" + '
  '0.049*"brain" + 0.037*"graphic" + 0.032*"convert" + 0.031*"upgrade" + '
  '0.029*"vote" + 0.023*"helmet"'),
 (4,
  '0.100*"key" + 0.072*"chip" + 0.036*"phone" + 0.034*"technology" + '
  '0.030*"com" + 0.029*"algorithm" + 0.027*"master" + 0.026*"encryption" + '
  '0.025*"tap" + 0.021*"government"'),
 (5,
  '0.046*"use" + 0.040*"system" + 0.025*"program" + 0.023*"thank" + '
  '0.022*"run" + 0.0

In [ ]:
lda_model.

In [35]:
doc_lda = lda_model[corpus]

In [36]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.826223980821485

Coherence Score:  0.41038999751791644


In [37]:
# Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.353437  0.016071       1        1  39.659615
0      0.278024 -0.157794       2        1  14.244300
2      0.260649  0.200164       3        1  12.890635
5      0.260186  0.121428       4        1   9.301243
19     0.146281 -0.036617       5        1   4.652315
18     0.082302 -0.340028       6        1   2.910666
14    -0.059364 -0.028929       7        1   1.888116
10    -0.031448  0.128053       8        1   1.739095
4     -0.036660  0.016097       9        1   1.579782
11    -0.085640 -0.045699      10        1   1.495224
16    -0.072486  0.044880      11        1   1.470775
13    -0.096583  0.018347      12        1   1.265691
8     -0.080078  0.015832      13        1   1.219233
7     -0.110434 -0.004299      14        1   1.159060
3     -0.118005  0.006945      15        1   1.018185
15    -0.113534 -0.013911      16        1   0.971010
1     -0.115524  0.025165      17        1   0.903749
17    -0.143545  0.014238      18        1   0.636075
6     -0.151344  0.006402      19        1   0.578835
12    -0.166234  0.013655      20        1   0.416391, topic_info=            Term          Freq         Total Category  logprob  loglift
140       people   7480.000000   7480.000000  Default  30.0000  30.0000
22    nntp_poste   3204.000000   3204.000000  Default  29.0000  29.0000
171          use   6071.000000   6071.000000  Default  28.0000  28.0000
517         host   2235.000000   2235.000000  Default  27.0000  27.0000
197        write  10857.000000  10857.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
9668        toss      0.048392      1.298660  Topic20 -11.4239   2.1915
6681      sphere      0.048393      1.298704  Topic20 -11.4239   2.1915
5249        scan      0.048395      1.298807  Topic20 -11.4238   2.1915
4017       color      0.048394      1.298779  Topic20 -11.4238   2.1915
3580        edge      0.048393      1.298721  Topic20 -11.4239   2.1915

[809 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2133     14  0.989633       abandon
37       18  0.963593  acceleration
1250      2  0.998675        accept
82        4  0.997443        access
742      20  0.954216     accessory
...     ...       ...           ...
197       1  0.999921         write
397      12  0.989917        writer
36        1  0.164791          year
36        3  0.693064          year
36        8  0.141877          year

[993 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 3, 6, 20, 19, 15, 11, 5, 12, 17, 14, 9, 8, 4, 16, 2, 18, 7, 13])